In [2]:
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
from tensorflow.keras.models import Sequential


from google.colab import drive
import numpy as np
import pandas as pd

In [3]:
pip install google.colab

     |████████████████████████████████| 81kB 2.7MB/s eta 0:00:01
     |████████████████████████████████| 71kB 7.1MB/s eta 0:00:011
     |████████████████████████████████| 112kB 12.8MB/s eta 0:00:01
     |████████████████████████████████| 768kB 14.7MB/s eta 0:00:01
     |████████████████████████████████| 8.0MB 1.2MB/s eta 0:00:011
     |████████████████████████████████| 15.9MB 2.0MB/s eta 0:00:01
     |████████████████████████████████| 61kB 2.8MB/s eta 0:00:011
     |████████████████████████████████| 491kB 2.3MB/s eta 0:00:01
     |████████████████████████████████| 245kB 2.1MB/s eta 0:00:01
     |████████████████████████████████| 122kB 1.8MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/14/2c/cd551d81dbe15200be1cf41cd03869a46fe7226e7450af7a6545bfc474c9/idna-2.8-py2.py3-none-any.whl
  Stored in directory: /Users/liuhao/Library/Caches/pip/wheels/38/0d/59/701e300a337b2a2e07b27fe74dbfff0bc56ac58f711566ee67
  Stored in directory: /Users/liuhao/Library/Caches/pip/wheels

In this example, we're going to train a [CharRNN](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) on a body of Shakespearian text. Ultimtely, this is an unsuperived learning task. But similar to our previous explorations in unsupervised DL, we will use an unlabeled dataset and create many samples of labeled data that we can use with our familiar supervised loss functions. The result will be a model that has learned the statistical properties of the input text, and can then be considered a "generative" model of language because we can use it to generate synthetic passages of Shakespeare.  

In [5]:
file_path = "./shakespeare.txt"

with open(file_path,"r") as f:
    text = f.read()

We've loaded our Shakespeare text, let's take a look at a random snippet.

In [6]:
print(text[31600:32000])

 lies i' the second chamber?
  LADY MACBETH. Donalbain.
  MACBETH. This is a sorry sight.           [Looks on his hands.
  LADY MACBETH. A foolish thought, to say a sorry sight.
  MACBETH. There's one did laugh in 's sleep, and one cried,
      "Murther!"
    That they did wake each other. I stood and heard them,
    But they did say their prayers and address'd them
    Again to sleep.
  LADY MACB


We need to convert our text into numeric arrays, the next several blocks accomplish this.

First, we'll create a mapping between characters and their numeric index. We'll also create the reverse mapping, which is useful.

In [7]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 75


Next, we'll create a training set of sub-sequences. Remember, we're trying to train a model to be able to predict the next chracter if it is given several characters of a subsequence. So we will create training pairs where each X is a fixed-length subsequences and each Y is the corresponding next letter in the text.

In [8]:
maxlen = 40
step = 3
sub_sequences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sub_sequences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sub_sequences))

nb sequences: 38700


In [9]:
k=300
print("(Sequence):\n" + sub_sequences[k])
print("\n(Target Character): \n" + next_chars[k])

(Sequence):
 and other Apparitions
  Lords, Gentleme

(Target Character): 
n


Next we'll create one-hot vectors for our sub-sequences. The tensor we create here will be shaped as (num_sequences x sequence_length x alphabet_size).

In [10]:
X = np.zeros((len(sub_sequences), maxlen, len(chars)), dtype=np.uint8 )
Y = np.zeros((len(sub_sequences), len(chars)), dtype=np.uint8)
for i, seq in enumerate(sub_sequences):
    for t, char in enumerate(seq):
        X[i, t, char_indices[char]] = 1
        Y[i, char_indices[next_chars[i]]] = 1

In [11]:
X[0,0,:]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=uint8)

In [12]:
Y[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

Our RNN model will be quite simple.

In [13]:
char_rnn = Sequential()
char_rnn.add(tfkl.LSTM(128, input_shape=(maxlen, len(chars))))
char_rnn.add(tfkl.Dense(len(chars),activation="softmax"))

In [14]:
char_rnn.compile(loss='categorical_crossentropy', optimizer=tfk.optimizers.RMSprop(lr=0.01))

In [15]:
char_rnn.fit(X,Y, epochs=20, batch_size=1024)

Epoch 1/20
38/38 [==============================] - 10s 263ms/step - loss: 3.2365
Epoch 2/20
38/38 [==============================] - 10s 253ms/step - loss: 2.7947
Epoch 3/20
38/38 [==============================] - 10s 250ms/step - loss: 2.3645
Epoch 4/20
38/38 [==============================] - 10s 252ms/step - loss: 2.0886
Epoch 5/20
38/38 [==============================] - 10s 252ms/step - loss: 1.9325
Epoch 6/20
38/38 [==============================] - 10s 251ms/step - loss: 1.8189
Epoch 7/20
38/38 [==============================] - 10s 253ms/step - loss: 1.7277
Epoch 8/20
38/38 [==============================] - 10s 253ms/step - loss: 1.6476
Epoch 9/20
38/38 [==============================] - 10s 252ms/step - loss: 1.5803
Epoch 10/20
38/38 [==============================] - 10s 251ms/step - loss: 1.5265
Epoch 11/20
38/38 [==============================] - 10s 251ms/step - loss: 1.4714
Epoch 12/20
38/38 [==============================] - 10s 252ms/step - loss: 1.4264
Epoch 13/20
3

Once we have a trained model, we can simulate new text by making predictions about the next character and then drawing characters in proportion to the predicted probabilities. And then simple repeat that process over and over, each time drawing the next character.

In [16]:
def draw_char(probs):
    probs = np.asarray(probs).astype('float64')
    if sum(probs) != 1.0:
        probs = probs / np.sum(probs)
    draw = np.random.choice(range(len(probs)) , p=probs)
    return draw

def sample_text(model, sample_length=100):
    start = np.random.randint(0, len(text) - maxlen - 1)
    sequence = text[start: start + maxlen]
  
    x_preds = np.zeros((sample_length, maxlen, len(chars)))
    for i in range(sample_length):
        for t, char in enumerate(sequence[-maxlen:]):
            x_preds[i, t, char_indices[char]] = 1.

        preds = model.predict(np.expand_dims(x_preds[i,:,:], axis=0), verbose=0)[0]
        next_index = draw_char(preds)
        next_char = indices_char[next_index]

        sequence += next_char
    return sequence

In [17]:
sim = sample_text(char_rnn,sample_length=500) 

In [18]:
print(sim)

at's shot
    Hath not yet lighted, and we be well to pe
    I sen seme to all we his haven but of but to now.
    Of keill from the rast hade of wonde.
    The loring of forreiss to you an y! Mecteld,
s ull coundaus, horrous with munt his haver!                     Entere. Slook to have lard to I, lo, mes fuld to their bent words,
    But not swanged at to come no youlp.
    Still hawe a hame Gon, honal that my and
    The spot to is's flets agat Is't smeet in her fion
    I bark not vealy but this I have lid do more.
    With 'tim w


Notice that we can do pretty well to learn the typical statistical patterns of this text and then simulate new text that appears to be very similar to legitimate Shakespeare. 

But just a caution - we can also do pretty well with a much simpler method (Markov model): http://nbviewer.jupyter.org/gist/yoavg/d76121dfde2618422139

So the lesson is to try something simple before jumping right in to deep learning.

## Exercise

In this example, we're going to use an RNN for sequence classification. The task we'll set up is to generate a training set of randomized strings, and train our model to detect whether a string contains any vowels.

First, we'll create a training dataset of short randomized character sequences and the corresponding label of whether or not they contain at least one vowel.

In [19]:
import string


In [20]:
def contains_vowels(sequence):
    vowels = ["a", "e", "i", "o", "u"]
    return any([vowel in list(sequence) for vowel in vowels])

In [21]:
contains_vowels("gradient")

True

In [22]:
sequences = []
labels = []
for i in range(1000):
    char_list = np.random.choice( list(string.ascii_lowercase), size = 5, replace=True)
    seq = "".join(char_list)
    sequences.append(seq)
    labels.append(int(contains_vowels(seq)))

In [23]:
df = pd.DataFrame({"sequence": sequences, "label":labels})

In [25]:
pip install pandas.compat

ERROR: Could not install packages due to an EnvironmentError: [Errno 2] No such file or directory: '/Users/liuhao/anaconda3/lib/python3.7/site-packages/pandas-1.0.3.dist-info/METADATA'

Note: you may need to restart the kernel to use updated packages.


In [26]:
df.head()

ImportError: cannot import name 'OrderedDict' from 'pandas.compat' (/Users/liuhao/anaconda3/lib/python3.7/site-packages/pandas/compat/__init__.py)

  sequence  label
0    qxmxq      0
1    wmiiu      1
2    wnqwi      1
3    rvegz      1
4    ujswg      1

Next, set up and train an RNN (of any type) to solve this task. What preprocessing will you need to do first on the raw data in order to prepare it for the network?

In [27]:
# Data Preprocessing
# your code here
char0 = list(string.ascii_lowercase)
char_indices0 = dict((c, i) for i, c in enumerate(char0))
indices_char0 = dict((i, c) for i, c in enumerate(char0))
X = np.zeros((len(sequences),5,26), dtype=np.uint8)
for i, seq in enumerate(sequences):
    for t,char in enumerate(seq):
        X[i,t,char_indices0[char]] = 1

In [28]:
# Model setup and training
# your code here
myrnn = Sequential()
myrnn.add(tfkl.LSTM(128, input_shape=(5,26)))
myrnn.add(tfkl.Dense(1,activation="sigmoid"))
myrnn.compile(loss='binary_crossentropy', optimizer=tfk.optimizers.RMSprop(lr=0.01),metrics=['accuracy'])
myrnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               79360     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 79,489
Trainable params: 79,489
Non-trainable params: 0
_________________________________________________________________


In [29]:
Y = np.array(labels)
results = myrnn.fit(X,Y,epochs=20,steps_per_epoch=20,batch_size=500)


Epoch 1/20
20/20 [==============================] - 0s 15ms/step - loss: 0.5502 - accuracy: 0.8168
Epoch 2/20
20/20 [==============================] - 0s 15ms/step - loss: 0.1656 - accuracy: 0.9527
Epoch 3/20
 0/20 [..............................] - 0s 0s/step - loss: 0.1656 - accuracy: 0.9527


In [ ]:
## idk why I do not have enough data, but will try to figure out later